In [6]:
#final update 2024-12-11
import sys
import os
import numpy as np
import scipy.stats as stats
import pandas as pd
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import seaborn as sb
import random as rd
import matplotlib
import warnings
import tqdm

In [ ]:
##detection frequency of thoes features with hits, include the biotransformation product and perform ms1 search again. in R
##calculate standard deviations and covariance of those features among 200 serum samples
##generate the final inclusion list for ms/ms acquisition
##with the ms2, perform molecular networking to support the plastic and their biotrnasformation products

In [ ]:
# Load peak area data for positive ion mode after batch correction
pos_peak_area_200 = pd.read_csv("D:/UCSF_postdoc_topic/REVEAL_topics/REVEAL_200samples_analysis/Pos_AlignmentResults/filtered_peak_area_for_bc_70.csv")

# Load target mz data
targetmzdat_pos = pd.read_csv('D:/UCSF_postdoc_topic/REVEAL_topics/REVEAL_200samples_analysis/Plastic_map_chemicallist_match/Plastic_Chemical_List_organic.csv')
#targetmzdat_pos = pd.read_csv('D:/UCSF_postdoc_topic/REVEAL_topics/REVEAL_200samples_analysis/Plastic_map_chemicallist_match/Plastic_Chemical_List_organic_withRTprediction.csv')
targetmzdat_pos['mz'] = targetmzdat_pos['Monoisotopic_Mass_ready'] + 1.007825

# Select relevant columns for inquiry
peak_area_inquiry = pos_peak_area_200[['Average Rt(min)', 'Average Mz'] + [col for col in pos_peak_area_200.columns if col.startswith('BH')]]

# Function to calculate ppm difference
def ppm_difference(value1, value2):
    return abs(value1 - value2) / value1 * 1e6

ppm_threshold = 5
matched_rows_pos = []
matched_summary = []

# MS1 search and matching
for _, chem_row in tqdm(targetmzdat_pos.iterrows(), total=targetmzdat_pos.shape[0], desc="Matching compounds"):
    mw = chem_row['mz']
    matches = peak_area_inquiry[peak_area_inquiry['Average Mz'].apply(lambda mz: ppm_difference(mz, mw) <= ppm_threshold)]
    if not matches.empty:
        # Append the matches along with the corresponding compound information
        for _, match_row in matches.iterrows():
            matched_rows_pos.append({
                **match_row,
                'Matched Compound': chem_row['SMILES'],
                'PREFERRED NAME': chem_row.get('PREFERRED NAME', None),
                'DTXSID_Hits': chem_row.get('DTXSID', None),
                'SMILES_STD': chem_row.get('SMILES_ready', None),
                'DTXSID': chem_row.get('DTXSID', None),
                'Name': chem_row.get('NAME', None),
                'Preferred Name': chem_row.get('PREFERRED NAME', None),
                'InChiKey_origin': chem_row.get('INCHIKEY', None)
            })
        # Record summary statistics for each targeted mz, preserving multiple hits
        matched_summary.append({
            'Targeted MZ': mw,
            'Matched Count': len(matches),
            'Average Rt(min)': matches['Average Rt(min)'].tolist(),
            'Average Mz': matches['Average Mz'].tolist(),
            'Mean Value': matches.iloc[:, 3:].mean(axis=1).tolist(),
            'Max Value': matches.iloc[:, 3:].max(axis=1).tolist(),
            'Min Value': matches.iloc[:, 3:].min(axis=1).tolist(),
            'Std Dev': matches.iloc[:, 3:].std(axis=1).tolist(),
            'RSD (%)': ((matches.iloc[:, 3:].std(axis=1) / matches.iloc[:, 3:].mean(axis=1)) * 100).tolist(),
            'SMILES_STD': chem_row.get('SMILES_ready', None),
            'DTXSID': chem_row.get('DTXSID', None),
            'CASRN': chem_row.get('CASRN', None),
            'Compound Name': chem_row.get('PREFERRED_NAME', None),
            'QC_NOTES': chem_row.get('QC_NOTES', None),
            'InChiKey_origin': chem_row.get('INCHIKEY', None)
        })

# Create a DataFrame for matched MSDIAL result rows
matched_peak_area_df_pos = pd.DataFrame(matched_rows_pos)
print(matched_peak_area_df_pos.shape)
# Create a DataFrame for matched summary statistics
matched_summary_df_pos = pd.DataFrame(matched_summary)
matched_summary_df_pos.to_csv("D:/UCSF_postdoc_topic/REVEAL_topics/REVEAL_200samples_analysis/Pos_AlignmentResults/mathced_targetedmz_summary_forallsamples_pos_woRT.csv")
print(matched_summary_df_pos.shape)

# Step 3: Handling technical duplicates and calculating mean values
bh_columns = [col for col in matched_peak_area_df_pos.columns if col.startswith('BH')]
mean_columns = {}
for col in bh_columns:
    base_col = col.split('_')[0]
    if base_col not in mean_columns:
        duplicate_cols = [c for c in bh_columns if c.startswith(base_col)]
        mean_columns[base_col] = matched_peak_area_df_pos[duplicate_cols].mean(axis=1)

# Retain only the columns with calculated mean values
matched_peak_area_df_pos = pd.concat([matched_peak_area_df_pos.drop(columns=bh_columns), pd.DataFrame(mean_columns)], axis=1)

# Step 4: Retain columns with median values > 5000, filter non-numeric values, and perform DataFrame transpose
filtered_df = matched_peak_area_df_pos.apply(pd.to_numeric, errors='coerce')  # Convert non-numeric values to NaN
filtered_df = filtered_df.loc[:, (filtered_df.median(axis=0) > 5000)].dropna(how='all', axis=1)  # Retain columns with median value > 5000

# Count the number of retained columns
num_retained_columns = filtered_df.shape[1]
print(f"Number of columns retained after filtering: {num_retained_columns}")

# Calculate summary statistics for each sample column
summary_stats = {
    'Sum': filtered_df.sum(),
    'Median': filtered_df.median(),
    'Mean': filtered_df.mean(),
    'Max': filtered_df.max(),
    'Min': filtered_df.min(),
    '95th Percentile': filtered_df.quantile(0.95),
    '5th Percentile': filtered_df.quantile(0.05)
}

# Create a DataFrame from the summary statistics
summary_stats_df_pos = pd.DataFrame(summary_stats)
summary_stats_df_pos.to_csv("D:/UCSF_postdoc_topic/REVEAL_topics/REVEAL_200samples_analysis/Pos_AlignmentResults/mathced_peaks_summary_forallsamples_pos_woRT.csv")

# Transpose the summary statistics DataFrame
# transposed_summary_df = summary_stats_df_pos.transpose()
# Display the final DataFrame
# print(summary_stats_df_pos)

C:\Users\yangj\AppData\Local\Temp\ipykernel_25660\1381856669.py:8: DtypeWarning: Columns (18,19,20,23,59,60) have mixed types. Specify dtype option on import or set low_memory=False.
  targetmzdat_pos = pd.read_csv('D:/UCSF_postdoc_topic/REVEAL_topics/REVEAL_200samples_analysis/Plastic_map_chemicallist_match/Plastic_Chemical_List_organic.csv')
Matching compounds: 100%|██████████| 5144/5144 [00:36<00:00, 140.79it/s]


(3974, 410)
(1121, 15)
Number of columns retained after filtering: 104


In [ ]:
##apply predicted RT
import pandas as pd
from tqdm import tqdm

# Load peak area data for positive ion mode after batch correction
pos_peak_area_200 = pd.read_csv("D:/UCSF_postdoc_topic/REVEAL_topics/REVEAL_200samples_analysis/Pos_AlignmentResults/filtered_peak_area_after_bc_70.csv")

# Load target mz data
# targetmzdat_pos = pd.read_csv('D:/UCSF_postdoc_topic/REVEAL_topics/REVEAL_200samples_analysis/Plastic_map_chemicallist_match/Plastic_Chemical_List_organic.csv')
targetmzdat_pos = pd.read_csv('D:/UCSF_postdoc_topic/REVEAL_topics/REVEAL_200samples_analysis/Plastic_map_chemicallist_match/Plastic_Chemical_List_organic_withRTprediction.csv')
targetmzdat_pos['mz'] = targetmzdat_pos['Monoisotopic_Mass_ready'] + 1.007825

# Select relevant columns for inquiry
peak_area_inquiry = pos_peak_area_200[['Average Rt(min)', 'Average Mz'] + [col for col in pos_peak_area_200.columns if col.startswith('BH')]]

# Function to calculate ppm difference
def ppm_difference(value1, value2):
    return abs(value1 - value2) / value1 * 1e6

def rt_diff(value1, valu2):
    return (value1-valu2)

ppm_threshold = 5
matched_rows_pos = []
matched_summary = []

# MS1 search and matching
for _, chem_row in tqdm(targetmzdat_pos.iterrows(), total=targetmzdat_pos.shape[0], desc="Matching compounds"):
    mw = chem_row['mz']
    rtlow = chem_row['Lower Bound PI']
    rthigh = chem_row['Upper Bound PI']
    matches = peak_area_inquiry[peak_area_inquiry['Average Mz'].apply(lambda mz: ppm_difference(mz, mw) <= ppm_threshold)]
    #print(f"Matches after ppm filtering: {matches.shape}")  # Debugging
    if matches.empty:
        continue
    matches = matches[matches['Average Rt(min)'].apply(lambda rt: rt_diff(rt, rtlow) >=0)]
    #print(f"Matches after Rt lower filter:: {matches.shape}")  # Debugging
    if matches.empty:
        continue
    matches = matches[matches['Average Rt(min)'].apply(lambda rt: rt_diff(rt, rthigh) <=0)]
    #print(f"Matches after Rt high filter:: {matches.shape}")  # Debugging
    if matches.empty:
        continue

    if not matches.empty:
        # Append the matches along with the corresponding compound information
        for _, match_row in matches.iterrows():
            matched_rows_pos.append({
                **match_row,
                'Matched Compound': chem_row['SMILES'],
                'PREFERRED NAME': chem_row.get('PREFERRED_NAME', None),
                'DTXSID_Hits': chem_row.get('DTXSID', None),
                'SMILES_STD': chem_row.get('SMILES_ready', None),
                'DTXSID': chem_row.get('DTXSID', None),
                'Name': chem_row.get('CASRN', None),
                'QC_NOTES': chem_row.get('QC_NOTES', None),
                'InChiKey_origin': chem_row.get('INCHIKEY', None)
            })
        # Record summary statistics for each targeted mz, preserving multiple hits
        matched_summary.append({
            'Targeted MZ': mw,
            'Matched Count': len(matches),
            'Average Rt(min)': matches['Average Rt(min)'].tolist(),
            'Average Mz': matches['Average Mz'].tolist(),
            'Mean Value': matches.iloc[:, 3:].mean(axis=1).tolist(),
            'Max Value': matches.iloc[:, 3:].max(axis=1).tolist(),
            'Min Value': matches.iloc[:, 3:].min(axis=1).tolist(),
            'Std Dev': matches.iloc[:, 3:].std(axis=1).tolist(),
            'RSD (%)': ((matches.iloc[:, 3:].std(axis=1) / matches.iloc[:, 3:].mean(axis=1)) * 100).tolist(),
            'SMILES_STD': chem_row.get('SMILES_ready', None),
            'DTXSID': chem_row.get('DTXSID', None),
            'CASRN': chem_row.get('CASRN', None),
            'Compound Name': chem_row.get('PREFERRED_NAME', None),
            'QC_NOTES': chem_row.get('QC_NOTES', None),
            'InChiKey_origin': chem_row.get('INCHIKEY', None)
        })

# Create a DataFrame for matched MSDIAL result rows
matched_peak_area_df_pos = pd.DataFrame(matched_rows_pos)
print(matched_peak_area_df_pos.shape)
# Create a DataFrame for matched summary statistics
matched_summary_df_pos = pd.DataFrame(matched_summary)
matched_summary_df_pos.to_csv("D:/UCSF_postdoc_topic/REVEAL_topics/REVEAL_200samples_analysis/Pos_AlignmentResults/mathced_targetedmz_summary_forallsamples_pos.csv")
print(matched_summary_df_pos.shape)

# Step 3: Handling technical duplicates and calculating mean values
bh_columns = [col for col in matched_peak_area_df_pos.columns if col.startswith('BH')]
mean_columns = {}
for col in bh_columns:
    base_col = col.split('_')[0]
    if base_col not in mean_columns:
        duplicate_cols = [c for c in bh_columns if c.startswith(base_col)]
        mean_columns[base_col] = matched_peak_area_df_pos[duplicate_cols].mean(axis=1)

# Retain only the columns with calculated mean values
matched_peak_area_df_pos = pd.concat([matched_peak_area_df_pos.drop(columns=bh_columns), pd.DataFrame(mean_columns)], axis=1)

# Step 4: Retain columns with median values > 5000, filter non-numeric values, and perform DataFrame transpose
filtered_df = matched_peak_area_df_pos.apply(pd.to_numeric, errors='coerce')  # Convert non-numeric values to NaN
filtered_df = filtered_df.loc[:, (filtered_df.median(axis=0) > 5000)].dropna(how='all', axis=1)  # Retain columns with median value > 5000

# Count the number of retained columns
num_retained_columns = filtered_df.shape[1]
print(f"Number of columns retained after filtering: {num_retained_columns}")

# Calculate summary statistics for each sample column
summary_stats = {
    'Sum': filtered_df.sum(),
    'Median': filtered_df.median(),
    'Mean': filtered_df.mean(),
    'Max': filtered_df.max(),
    'Min': filtered_df.min(),
    '95th Percentile': filtered_df.quantile(0.95),
    '5th Percentile': filtered_df.quantile(0.05)
}

# Create a DataFrame from the summary statistics
summary_stats_df_pos = pd.DataFrame(summary_stats)
summary_stats_df_pos.to_csv("D:/UCSF_postdoc_topic/REVEAL_topics/REVEAL_200samples_analysis/Pos_AlignmentResults/mathced_peaks_summary_forallsamples_pos.csv")

# Transpose the summary statistics DataFrame
# transposed_summary_df = summary_stats_df_pos.transpose()
# Display the final DataFrame
# print(summary_stats_df_pos)

C:\Users\yangj\AppData\Local\Temp\ipykernel_25660\3624809644.py:10: DtypeWarning: Columns (18,19,20,23,59,60) have mixed types. Specify dtype option on import or set low_memory=False.
  targetmzdat_pos = pd.read_csv('D:/UCSF_postdoc_topic/REVEAL_topics/REVEAL_200samples_analysis/Plastic_map_chemicallist_match/Plastic_Chemical_List_organic_withRTprediction.csv')
Matching compounds: 100%|██████████| 5144/5144 [01:26<00:00, 59.16it/s] 


(1340, 410)
(535, 15)
Number of columns retained after filtering: 122


In [57]:
#rank the feature by rt AND mz
matched_peak_pos = matched_peak_area_df_pos.sort_values(by=['Average Rt(min)', 'Average Mz'])
#filerted features with intensity lower than 5000 
peak_area = matched_peak_pos[[col for col in matched_peak_pos.columns if col.startswith('BH')]]
row_medians = peak_area.median(axis=1)
peak_index = row_medians[row_medians >= 5000].index
filter_results = matched_peak_pos.loc[peak_index]
filter_results.shape
filter_results.to_csv("D:/UCSF_postdoc_topic/REVEAL_topics/REVEAL_200samples_analysis/Pos_AlignmentResults/matched_peak_pos_over5000.csv")

In [50]:
matched_peak_pos.to_csv("D:/UCSF_postdoc_topic/REVEAL_topics/REVEAL_200samples_analysis/Pos_AlignmentResults/matched_peak_pos.csv")
#splitting the target list


In [35]:
#Negative alignment results
from tqdm import tqdm
# Load peak area data for positive ion mode after batch correction
# pos_peak_area_200 = pd.read_csv("D:/UCSF_postdoc_topic/REVEAL_topics/REVEAL_200samples_analysis/Pos_AlignmentResults/filtered_peak_area_after_bc.csv")
# neg_peak_area_200 = pd.read_csv("D:/UCSF_postdoc_topic/REVEAL_topics/REVEAL_200samples_analysis/Neg_AlignmentResults/filtered_peak_area_after_bc_70.csv")
neg_peak_area_200 = pd.read_csv("D:/UCSF_postdoc_topic/REVEAL_topics/REVEAL_200samples_analysis/Neg_AlignmentResults/filtered_peak_area_for_bc_70.csv")
# Load target mz data
targetmzdat_neg = pd.read_csv('D:/UCSF_postdoc_topic/REVEAL_topics/REVEAL_200samples_analysis/plastic_map_chemlist/Plastic_Chemical_List_organic_withRTprediction_new_SMILES.csv')
targetmzdat_neg['mz'] = targetmzdat_neg['MONOISOTOPIC_MASS'] - 1.007825

# Select relevant columns for inquiry
peak_area_inquiry = neg_peak_area_200[['Average Rt(min)', 'Average Mz'] + [col for col in neg_peak_area_200.columns if col.startswith('BH')]]

# Function to calculate ppm difference
def ppm_difference(value1, value2):
    return abs(value1 - value2) / value1 * 1e6

ppm_threshold = 5
matched_rows_neg = []
matched_summary = []

# MS1 search and matching
for _, chem_row in tqdm(targetmzdat_neg.iterrows(), total=targetmzdat_neg.shape[0], desc="Matching compounds"):
    mw = chem_row['mz']
    matches = peak_area_inquiry[peak_area_inquiry['Average Mz'].apply(lambda mz: ppm_difference(mz, mw) <= ppm_threshold)]
    #print(f"Matches after ppm filtering: {matches.shape}")  # Debugging
    
    if not matches.empty:
        # Append the matches along with the corresponding compound information
        for _, match_row in matches.iterrows():
            matched_rows_neg.append({
                **match_row,
                'Matched Compound': chem_row['SMILES'],
                'PREFERRED_NAME': chem_row.get('PREFERRED_NAME', None),
                'DTXSID_Hits': chem_row.get('DTXSID', None),
                'DTXSID': chem_row.get('DTXSID', None),
                'InChiKey_origin': chem_row.get('INCHIKEY', None),
                'MOLECULAR_FORMULA_original': chem_row.get('MOLECULAR_FORMULA', None),
                'Compound_comment': chem_row.get('QC_NOTES', None),
                'SMILES_STD': chem_row.get('SMILES_ready', None),
                "Monoisotopic_mass_ready": chem_row.get('Monoisotopic_Mass_ready', None),
                "PredictedRT": chem_row.get('Predicted RT', None),
                "PredictedRT_lower": chem_row.get('Lower Bound PI', None),
                "PredictedRT_upper": chem_row.get('Upper Bound PI', None),
                'ENTACT_STD_RT' : chem_row.get('RT_ENTACT', None),
                'DTSC_STD_RT': chem_row.get('RT_DTSC', None)
            })
        # Record summary statistics for each targeted mz, preserving multiple hits
        matched_summary.append({
            'Targeted MZ': mw,
            'Matched Count': len(matches),
            'Average Rt(min)': matches['Average Rt(min)'].tolist(),
            'Average Mz': matches['Average Mz'].tolist(),
            'Mean Value': matches.iloc[:, 3:].mean(axis=1).tolist(),
            'Max Value': matches.iloc[:, 3:].max(axis=1).tolist(),
            'Min Value': matches.iloc[:, 3:].min(axis=1).tolist(),
            'Std Dev': matches.iloc[:, 3:].std(axis=1).tolist(),
            'RSD (%)': ((matches.iloc[:, 3:].std(axis=1) / matches.iloc[:, 3:].mean(axis=1)) * 100).tolist(),
            'SMILES_STD': chem_row.get('SMILES_ready', None),
            'DTXSID': chem_row.get('DTXSID', None),
            'CASRN': chem_row.get('CASRN', None),
            'Compound Name': chem_row.get('PREFERRED_NAME', None),
            'QC_NOTES': chem_row.get('QC_NOTES', None),
            'InChiKey_origin': chem_row.get('INCHIKEY', None)
        })

# Create a DataFrame for matched MSDIAL result rows
matched_peak_area_df_neg = pd.DataFrame(matched_rows_neg)
print(matched_peak_area_df_neg.shape)

# Create a DataFrame for matched summary statistics
matched_summary_df_neg = pd.DataFrame(matched_summary)
print(f"#of targetedmz with hits:{matched_summary_df_neg.shape}")
# matched_summary_df_neg.to_csv("D:/UCSF_postdoc_topic/REVEAL_topics/REVEAL_200samples_analysis/Neg_AlignmentResults/mathced_targetedmz_summary_forallsamples_Neg_woRT.csv")
print(matched_summary_df_neg.shape)

# Step 3: Handling technical duplicates and calculating mean values
bh_columns = [col for col in matched_peak_area_df_neg.columns if col.startswith('BH')]
mean_columns = {}
for col in bh_columns:
    base_col = col.split('_')[0]
    if base_col not in mean_columns:
        duplicate_cols = [c for c in bh_columns if c.startswith(base_col)]
        mean_columns[base_col] = matched_peak_area_df_neg[duplicate_cols].mean(axis=1)

# Retain only the columns with calculated mean values
matched_peak_area_df_neg = pd.concat([matched_peak_area_df_neg.drop(columns=bh_columns), pd.DataFrame(mean_columns)], axis=1)

#output the matched peak area data
#rank the feature by rt AND mz
matched_peak_neg = matched_peak_area_df_neg.sort_values(by=['Average Rt(min)', 'Average Mz'])
#filerted features with intensity lower than 5000 
peak_areaneg = matched_peak_neg[[col for col in matched_peak_neg.columns if col.startswith('BH')]]
row_mediansneg = peak_areaneg.median(axis=1)
peak_indexneg = row_mediansneg[row_mediansneg >= 5000].index
filter_resultsneg = matched_peak_neg.loc[peak_indexneg]
print(filter_resultsneg.shape)
# filter_resultsneg.to_csv("D:/UCSF_postdoc_topic/REVEAL_topics/REVEAL_200samples_analysis/Neg_AlignmentResults/matched_peak_neg_over5000_wopredRT.csv")

# Step 4: Retain columns with median values > 5000, filter non-numeric values, and perform DataFrame transpose
filtered_df = matched_peak_area_df_neg.apply(pd.to_numeric, errors='coerce')  # Convert non-numeric values to NaN
filtered_df = filtered_df.loc[:, (filtered_df.median(axis=0) > 5000)].dropna(how='all', axis=1)  # Retain columns with median value > 5000

# Count the number of retained columns
num_retained_columns = filtered_df.shape[1]
print(f"Number of columns retained after filtering: {num_retained_columns}")

# Calculate summary statistics for each sample column for each individual samples
#
summary_stats = {
    'Sum': filtered_df.sum(),
    'Median': filtered_df.median(),
    'Mean': filtered_df.mean(),
    'Max': filtered_df.max(),
    'Min': filtered_df.min(),
    '95th Percentile': filtered_df.quantile(0.95),
    '5th Percentile': filtered_df.quantile(0.05)
}

# Create a DataFrame from the summary statistics
summary_stats_df_neg = pd.DataFrame(summary_stats)
# summary_stats_df_neg.to_csv("D:/UCSF_postdoc_topic/REVEAL_topics/REVEAL_200samples_analysis/Neg_AlignmentResults/mathced_peaks_summary_forallsamples_Neg_woRT.csv")

# Transpose the summary statistics DataFrame
# transposed_summary_df_neg = summary_stats_df_neg.transpose()
# Display the final DataFrame
# print(summary_stats_df_neg)

Matching compounds: 100%|██████████| 5144/5144 [00:45<00:00, 113.97it/s]


(8098, 416)
#of targetedmz with hits:(1507, 15)
(1507, 15)
(4302, 217)
Number of columns retained after filtering: 134


In [36]:
#Update on 20241211
#check stock in ENTACKT mixture
ENTACTmix = pd.read_csv("D:/UCSF_postdoc_topic/REVEAL_topics/REVEAL_200samples_analysis/plastic_map_chemlist/ENTACT_Mixture_chemlist.csv")
#add experimental RT to the matched peaks

#check std availability and make annotation
for iter, row in filter_resultsneg.iterrows():
    if row['DTXSID_Hits'] in ENTACTmix['DTXSID'].tolist():
        filter_resultsneg.loc[iter, 'ENTACT_std_availability'] = 'Yes'
        # filter_resultsneg.loc[iter, 'ENTACT_std_RT'] = 'ENTACT_std_RT'
    else:
        filter_resultsneg.loc[iter, 'ENTACT_std_availability'] = 'No'

filter_resultsneg.to_csv("D:/UCSF_postdoc_topic/REVEAL_topics/REVEAL_200samples_analysis/Neg_AlignmentResults/matched_peak_neg_over5000_wopredRT.csv")

In [ ]:
filter_resultsneg = pd.read_csv("D:/UCSF_postdoc_topic/REVEAL_topics/REVEAL_200samples_analysis/Neg_AlignmentResults/matched_peak_neg_over5000_wopredRT.csv")
#make the suspects with predictedRT upper > average RT > predictedRT lower as the fisrt round of targeted msms analysis 
#make the suspects outside the predicted RT range as the second round of the targeted msms analysis
First_round = filter_resultsneg[(filter_resultsneg['Average Rt(min)'] >= filter_resultsneg['PredictedRT_lower']) & (filter_resultsneg['Average Rt(min)'] <= filter_resultsneg['PredictedRT_upper'])]
Second_round = filter_resultsneg[(filter_resultsneg['Average Rt(min)'] < filter_resultsneg['PredictedRT_lower']) | (filter_resultsneg['Average Rt(min)'] > filter_resultsneg['PredictedRT_upper'])]

#import the first round results and cross check with the second round 
#add candidates with the same mz and rt from the seconde round to the first round 
print(First_round.shape)
print(Second_round.shape)

#update the first round with experimental rt from ENTACT RT library

#update the first round with the experimental Record from previous results
previous_first_round = pd.read_csv("D:/UCSF_postdoc_topic/REVEAL_topics/REVEAL_200samples_analysis/Neg_AlignmentResults/matched_peak_neg_over5000_firstround_msms.csv")
# # Frist_round_amend 
for iter, row in First_round.iterrows():
    match_mz = First_round['Average Mz']
    match_rt = First_round['Average Rt(min)']

    if row['DTXSID'] in previous_first_round['DTXSID'].tolist():
        First_round.loc[iter, 'Previous_MSMS'] = 'Yes'
    else:
        First_round.loc[iter, 'Previous_MSMS'] = 'No'
    


(1513, 218)
(2789, 218)


In [18]:
import pandas as pd
from tqdm import tqdm

# Load peak area data for positive ion mode after batch correction
# pos_peak_area_200 = pd.read_csv("D:/UCSF_postdoc_topic/REVEAL_topics/REVEAL_200samples_analysis/Pos_AlignmentResults/filtered_peak_area_after_bc.csv")
# neg_peak_area_200 = pd.read_csv("D:/UCSF_postdoc_topic/REVEAL_topics/REVEAL_200samples_analysis/Neg_AlignmentResults/filtered_peak_area_after_bc_70.csv")
neg_peak_area_200 = pd.read_csv("D:/UCSF_postdoc_topic/REVEAL_topics/REVEAL_200samples_analysis/Neg_AlignmentResults/filtered_peak_area_for_bc_70.csv")
# Load target mz data
targetmzdat_neg = pd.read_csv('D:/UCSF_postdoc_topic/REVEAL_topics/REVEAL_200samples_analysis/plastic_map_chemlist/Plastic_Chemical_List_organic_withRTprediction.csv')
targetmzdat_neg['mz'] = targetmzdat_neg['MONOISOTOPIC_MASS'] - 1.007825

# Select relevant columns for inquiry
peak_area_inquiry = neg_peak_area_200[['Average Rt(min)', 'Average Mz'] + [col for col in neg_peak_area_200.columns if col.startswith('BH')]]

# Function to calculate ppm difference
def ppm_difference(value1, value2):
    return abs(value1 - value2) / value1 * 1e6

def rt_diff(value1, valu2):
    return (value1-valu2)

ppm_threshold = 5
matched_rows_neg = []
matched_summary = []

# MS1 search and matching
for _, chem_row in tqdm(targetmzdat_neg.iterrows(), total=targetmzdat_neg.shape[0], desc="Matching compounds"):
    mw = chem_row['mz']
    rtlow = chem_row['Lower Bound PI']
    rthigh = chem_row['Upper Bound PI']
    matches = peak_area_inquiry[peak_area_inquiry['Average Mz'].apply(lambda mz: ppm_difference(mz, mw) <= ppm_threshold)]
    #print(f"Matches after ppm filtering: {matches.shape}")  # Debugging
    if matches.empty:
        continue
    matches = matches[matches['Average Rt(min)'].apply(lambda rt: rt_diff(rt, rtlow) >=0)]
    #print(f"Matches after Rt lower filter:: {matches.shape}")  # Debugging
    if matches.empty:
        continue
    matches = matches[matches['Average Rt(min)'].apply(lambda rt: rt_diff(rt, rthigh) <=0)]
    #print(f"Matches after Rt high filter:: {matches.shape}")  # Debugging
    if matches.empty:
        continue
    
    if not matches.empty:
        # Append the matches along with the corresponding compound information
        for _, match_row in matches.iterrows():
            matched_rows_neg.append({
                **match_row,
                'Matched Compound': chem_row['SMILES'],
                'PREFERRED NAME': chem_row.get('PREFERRED NAME', None),
                'DTXSID_Hits': chem_row.get('DTXSID', None),
                'SMILES_STD': chem_row.get('SMILES_ready', None),
                'DTXSID': chem_row.get('DTXSID', None),
                'Name': chem_row.get('NAME', None),
                'InChiKey_origin': chem_row.get('INCHIKEY', None)
            })
        # Record summary statistics for each targeted mz, preserving multiple hits
        matched_summary.append({
            'Targeted MZ': mw,
            'Matched Count': len(matches),
            'Average Rt(min)': matches['Average Rt(min)'].tolist(),
            'Average Mz': matches['Average Mz'].tolist(),
            'Mean Value': matches.iloc[:, 3:].mean(axis=1).tolist(),
            'Max Value': matches.iloc[:, 3:].max(axis=1).tolist(),
            'Min Value': matches.iloc[:, 3:].min(axis=1).tolist(),
            'Std Dev': matches.iloc[:, 3:].std(axis=1).tolist(),
            'RSD (%)': ((matches.iloc[:, 3:].std(axis=1) / matches.iloc[:, 3:].mean(axis=1)) * 100).tolist(),
            'SMILES_STD': chem_row.get('SMILES_ready', None),
            'DTXSID': chem_row.get('DTXSID', None),
            'CASRN': chem_row.get('CASRN', None),
            'Compound Name': chem_row.get('PREFERRED_NAME', None),
            'QC_NOTES': chem_row.get('QC_NOTES', None),
            'InChiKey_origin': chem_row.get('INCHIKEY', None)
        })

# Create a DataFrame for matched MSDIAL result rows
matched_peak_area_df_neg = pd.DataFrame(matched_rows_neg)
print(matched_peak_area_df_neg.shape)

# Create a DataFrame for matched summary statistics
matched_summary_df_neg = pd.DataFrame(matched_summary)
print(f"#of targetedmz with hits:{matched_summary_df_neg.shape}")
# matched_summary_df_neg.to_csv("D:/UCSF_postdoc_topic/REVEAL_topics/REVEAL_200samples_analysis/Neg_AlignmentResults/mathced_targetedmz_summary_forallsamples_Neg.csv")
print(matched_summary_df_neg.shape)

# Step 3: Handling technical duplicates and calculating mean values
bh_columns = [col for col in matched_peak_area_df_neg.columns if col.startswith('BH')]
mean_columns = {}
for col in bh_columns:
    base_col = col.split('_')[0]
    if base_col not in mean_columns:
        duplicate_cols = [c for c in bh_columns if c.startswith(base_col)]
        mean_columns[base_col] = matched_peak_area_df_neg[duplicate_cols].mean(axis=1)

# Retain only the columns with calculated mean values
matched_peak_area_df_neg = pd.concat([matched_peak_area_df_neg.drop(columns=bh_columns), pd.DataFrame(mean_columns)], axis=1)

#output the matched peak area data
#rank the feature by rt AND mz
matched_peak_neg = matched_peak_area_df_neg.sort_values(by=['Average Rt(min)', 'Average Mz'])
#filerted features with intensity lower than 5000 
peak_areaneg = matched_peak_neg[[col for col in matched_peak_neg.columns if col.startswith('BH')]]
row_mediansneg = peak_areaneg.median(axis=1)
peak_indexneg = row_mediansneg[row_mediansneg >= 5000].index
filter_resultsneg = matched_peak_neg.loc[peak_indexneg]
print(filter_resultsneg.shape)
# filter_resultsneg.to_csv("D:/UCSF_postdoc_topic/REVEAL_topics/REVEAL_200samples_analysis/Neg_AlignmentResults/matched_peak_neg_over5000_wopredRT.csv")

# Step 4: Retain columns with median values > 5000, filter non-numeric values, and perform DataFrame transpose
filtered_df = matched_peak_area_df_neg.apply(pd.to_numeric, errors='coerce')  # Convert non-numeric values to NaN
filtered_df = filtered_df.loc[:, (filtered_df.median(axis=0) > 5000)].dropna(how='all', axis=1)  # Retain columns with median value > 5000

# Count the number of retained columns
num_retained_columns = filtered_df.shape[1]
print(f"Number of columns retained after filtering: {num_retained_columns}")

# Calculate summary statistics for each sample column for each individual samples
summary_stats = {
    'Sum': filtered_df.sum(),
    'Median': filtered_df.median(),
    'Mean': filtered_df.mean(),
    'Max': filtered_df.max(),
    'Min': filtered_df.min(),
    '95th Percentile': filtered_df.quantile(0.95),
    '5th Percentile': filtered_df.quantile(0.05)
}

# Create a DataFrame from the summary statistics
summary_stats_df_neg = pd.DataFrame(summary_stats)
# summary_stats_df_neg.to_csv("D:/UCSF_postdoc_topic/REVEAL_topics/REVEAL_200samples_analysis/Neg_AlignmentResults/mathced_peaks_summary_forallsamples_Neg.csv")

# Transpose the summary statistics DataFrame
# transposed_summary_df_neg = summary_stats_df_neg.transpose()
# Display the final DataFrame
# print(summary_stats_df_neg)

C:\Users\yangj\AppData\Local\Temp\ipykernel_9172\456453554.py:9: DtypeWarning: Columns (18,19,20,23,59,60) have mixed types. Specify dtype option on import or set low_memory=False.
  targetmzdat_neg = pd.read_csv('D:/UCSF_postdoc_topic/REVEAL_topics/REVEAL_200samples_analysis/plastic_map_chemlist/Plastic_Chemical_List_organic_withRTprediction.csv')
Matching compounds: 100%|██████████| 5144/5144 [00:36<00:00, 140.09it/s]


(3006, 409)
#of targetedmz with hits:(781, 15)
(781, 15)
(1546, 210)
Number of columns retained after filtering: 106


In [ ]:
from rdkit import Chem
from rdkit.Chem import MolToSmiles
##cross refrence, cross check with blood exposome database
bloodexpo_dat = pd.read_csv("D:/UCSF_postdoc_topic/REVEAL_topics/references/blood_exposome_database/blood_exposome_chemicals_july_2023.csv")

# Step 1: Filter rows with available MW (MONOISOTOPIC MASS) between 100 and 1000 and classified as organic
bloodexpo_modified = []
for index, row in bloodexpo_dat.iterrows():
    smiles = row['CanonicalSMILES']  # Assuming the SMILES column is named 'SMILES'
    mol = Chem.MolFromSmiles(smiles)
    canonical_smiles = Chem.MolToSmiles(mol, canonical=True) if mol else None
    bloodexpo_modified.append({
        **row,
        "SMILES_ready": canonical_smiles})

# Create a new DataFrame from the filtered rows
bloodexpo_modified = pd.DataFrame(bloodexpo_modified)

# Match SMILES from match_summary_df with bloodexpo_modified and retain hits
# Match inchikey original between hits and the blood exposome database
# matched_hits_SMILES = pd.merge(matched_summary_df_neg, bloodexpo_modified, how='inner', left_on='SMILES_STD', right_on='SMILES_ready')
matched_hits_inchi = pd.merge(matched_summary_df_neg, bloodexpo_modified, how='inner', left_on='InChiKey_origin', right_on='InChIKey')
#print(matched_hits_SMILES.shape)
print(matched_hits_inchi.shape)
# Match inchikey original between hits and the blood exposome database
print(matched_summary_df_neg.shape)

#title_dict = matched_hits_SMILES.groupby('SMILES_STD')['Title'].apply(list).to_dict()
title_dict2 = matched_hits_inchi.groupby('InChiKey_origin')['Title'].apply(list).to_dict()
papercount_dict = matched_hits_inchi.groupby('InChiKey_origin')['BloodPaperCount'].apply(list).to_dict()
#matched_summary_df_neg['matched_with_bloodexpo_smi'] = matched_summary_df_neg['SMILES_STD'].map(title_dict).apply(lambda x: x if isinstance(x,list) else[])
matched_summary_df_neg['matched_with_bloodexpo_inchi'] = matched_summary_df_neg['InChiKey_origin'].map(title_dict2).apply(lambda x: x if isinstance(x,list) else[])
matched_summary_df_neg['BloodPaperCount'] = matched_summary_df_neg['InChiKey_origin'].map(papercount_dict).apply(lambda x: x if isinstance(x,list) else[])
matched_summary_df_neg.to_csv("D:/UCSF_postdoc_topic/REVEAL_topics/REVEAL_200samples_analysis/Neg_AlignmentResults/mathced_targetedmz_summary_forallsamples_Neg.csv")
##annotate the targeted mz with the function information and the production volume information from the EST paper
##when matching the plastic paper, some compounds are missing
# plasticmap_est = pd.read_csv('D:/UCSF_postdoc_topic/REVEAL_topics/plastic_related_chemicals/plasticmap_from_ESTpaper.csv',encoding='ISO-8859-1')
# # Match Inchikey from match_summary_df with bloodexpo_modified and retain hits
# matched_summary_df_neg_plastic_note = pd.merge(matched_summary_df_neg, plasticmap_est, how='inner', left_on='InChiKey_origin', right_on='InChI_key')
# matched_summary_df_neg_plastic_note.to_csv("D:/UCSF_postdoc_topic/REVEAL_topics/REVEAL_200samples_analysis/Neg_AlignmentResults/mathced_peaks_summary_withplasticannotation_Neg.csv")

(303, 35)
(781, 15)


In [ ]:
# Match SMILES from match_summary_df with bloodexpo_modified and retain hits
#matched_hits_SMILES = pd.merge(matched_summary_df_pos, bloodexpo_modified, how='inner', left_on='SMILES_STD', right_on='SMILES_ready')
matched_hits_inchi = pd.merge(matched_summary_df_pos, bloodexpo_modified, how='inner', left_on='InChiKey_origin', right_on='InChIKey')

# print(matched_hits_SMILES.shape)
print(matched_hits_inchi.shape)
# Match inchikey original between hits and the blood exposome database
print(matched_summary_df_pos.shape)

# title_dict = matched_hits_SMILES.groupby('SMILES_STD')['Title'].apply(list).to_dict()
title_dict2 = matched_hits_inchi.groupby('InChiKey_origin')['Title'].apply(list).to_dict()
papercount_dict = matched_hits_inchi.groupby('InChiKey_origin')['BloodPaperCount'].apply(list).to_dict()
matched_summary_df_pos['matched_with_bloodexpo_inchi'] = matched_summary_df_pos['InChiKey_origin'].map(title_dict2).apply(lambda x: x if isinstance(x,list) else[])
matched_summary_df_pos['BloodPaperCount'] = matched_summary_df_pos['InChiKey_origin'].map(papercount_dict).apply(lambda x: x if isinstance(x,list) else[])
matched_summary_df_pos.to_csv("D:/UCSF_postdoc_topic/REVEAL_topics/REVEAL_200samples_analysis/Pos_AlignmentResults/mathced_targetedmz_summary_forallsamples_Pos.csv")

##annotate the targeted mz with the function information and the production volume information from the EST paper
##when matching the plastic paper, some compounds are missing
# plasticmap_est = pd.read_csv('D:/UCSF_postdoc_topic/REVEAL_topics/plastic_related_chemicals/plasticmap_from_ESTpaper.csv',encoding='ISO-8859-1')
# # Match Inchikey from match_summary_df with bloodexpo_modified and retain hits
# matched_summary_df_pos_plastic_note = pd.merge(matched_summary_df_pos, plasticmap_est, how='inner', left_on='InChiKey_origin', right_on='InChI_key')
# matched_summary_df_pos_plastic_note.to_csv("D:/UCSF_postdoc_topic/REVEAL_topics/REVEAL_200samples_analysis/Pos_AlignmentResults/mathced_peaks_summary_withplasticannotation_pos.csv")

(210, 35)
(535, 15)


In [ ]:
######
##generate the files for CFM-ID spectra prediciton
# CFMID_ready_df = matched_summary_df_neg[['SMILES_STD']].copy()
# CFMID_ready_df['ID'] = ['Molecule' + str(i + 1) for i in range(len(CFMID_ready_df))]
# CFMID_ready_df = CFMID_ready_df[['ID','SMILES_STD']]
# ##CFMID_ready_df.to_csv('D:/UCSF_postdoc_topic/REVEAL_topics/First100_batch/neg_CFMID_MSready.txt', sep='\t', index=False, header=False)
# CFMID_ready_df.to_csv('D:/UCSF_postdoc_topic/REVEAL_topics/REVEAL_200samples_analysis/Neg_AlignmentResults/neg_CFMID_MSready.txt', sep='\t', index=False, header=False)

# #####
# #generate the files for CFM-ID spectra prediciton
# CFMID_ready_df = matched_summary_df_pos[['SMILES_STD']].copy()
# CFMID_ready_df['ID'] = ['Molecule' + str(i + 1) for i in range(len(CFMID_ready_df))]
# CFMID_ready_df = CFMID_ready_df[['ID','SMILES_STD']]
# # CFMID_ready_df.to_csv('D:/UCSF_postdoc_topic/REVEAL_topics/First100_batch/neg_CFMID_MSready.txt', sep='\t', index=False, header=False)
# CFMID_ready_df.to_csv('D:/UCSF_postdoc_topic/REVEAL_topics/REVEAL_200samples_analysis/Pos_AlignmentResults/Pos_CFMID_MSready.txt', sep='\t', index=False, header=False)

In [ ]:
import pickle
import os 

# Save all variables in the environment, excluding un-picklable ones
with open('D:/UCSF_postdoc_topic/REVEAL_topics/REVEAL_200samples_analysis/scripts/saved_environment.pkl', 'wb') as f:
    save_dict = {}
    for var_name, var_value in globals().items():
        if not var_name.startswith('__') and var_name != 'f':
            try:
                pickle.dumps(var_value)
                save_dict[var_name] = var_value
            except (pickle.PicklingError, AttributeError, TypeError):
                # Ignore un-picklable variables
                print(f"Variable '{var_name}' could not be pickled and was ignored.")
    pickle.dump(save_dict, f)

print("Environment saved successfully.")

Variable 'get_ipython' could not be pickled and was ignored.
Variable 'exit' could not be pickled and was ignored.
Variable 'quit' could not be pickled and was ignored.
Variable 'open' could not be pickled and was ignored.
Variable 'sys' could not be pickled and was ignored.
Variable 'os' could not be pickled and was ignored.
Variable 'np' could not be pickled and was ignored.
Variable 'stats' could not be pickled and was ignored.
Variable 'pd' could not be pickled and was ignored.
Variable 'plt' could not be pickled and was ignored.
Variable 'sb' could not be pickled and was ignored.
Variable 'rd' could not be pickled and was ignored.
Variable 'matplotlib' could not be pickled and was ignored.
Variable 'warnings' could not be pickled and was ignored.
Variable 'pickle' could not be pickled and was ignored.


In [39]:
import pickle
import os
import pandas as pd
import rdkit 
from tqdm import tqdm

def ppm_difference(value1, value2):
    return abs(value1 - value2) / value1 * 1e6

def rt_diff(value1, valu2):
    return (value1-valu2)

# Load all variables from the saved environment
with open('D:/UCSF_postdoc_topic/REVEAL_topics/REVEAL_200samples_analysis/scripts/saved_environment.pkl', 'rb') as f:
    loaded_env = pickle.load(f)

# Assign variables back to the global namespace
globals().update(loaded_env)

print("Environment loaded successfully.")

EOFError: Ran out of input